In [1]:
!hostname

gpu08.doc.ic.ac.uk


In [2]:
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pickle

from Oasis_DataModule_file import Oasis_PL
from UKBB_All_Substructs_Net_file import UKBB_All_Substructs_Net2, Oasis_PCA_Net, UKBB_All_Substructs_Net2_Binary

from sklearn.metrics import confusion_matrix
import os
import torch

In [3]:
from Oasis_DataModule_file import all_sub_structs_list
# substructs = ["L_Hipp", "R_Hipp"]
substructs = all_sub_structs_list

In [4]:
rand_seeds = [4, 8]
min_epochs = 5
max_epochs = 25

# Set filename & redo model and fit function (e.g. double etc)

In [5]:
folder = "./static_classification_results_binary_2/final_attempt"

In [6]:
models_filename_base = "GCN_all_subs_"

In [7]:
# results_filename = os.path.join(folder, results_filename)
predictions_filename_base = models_filename_base + "preds_"
models_filename_base = os.path.join(folder, models_filename_base)
predictions_filename_base = os.path.join(folder, predictions_filename_base)
# print("results: ", results_filename)
print("models: ", models_filename_base)
print("predis: ", predictions_filename_base)

models:  ./static_classification_results_binary_2/final_attempt/GCN_all_subs_
predis:  ./static_classification_results_binary_2/final_attempt/GCN_all_subs_preds_


In [8]:
for seed in rand_seeds:
    torch.cuda.empty_cache()
    
    #re-seed everything
    pl.seed_everything(seed, workers=True)
    
    #setup data (static labels type, rebalanced)
    pl_test = Oasis_PL(
        batch_size = 20,
        reload_path = False,
        balance_classes = True,
        static = True
    )
    
    # get umeyama working for data
    pl_test.prepare_data()
    pl_test.setup(reload_sampler = True)
    pl_test.set_umeyama(pl_test.train_set, n_examples = 100)

    #setup model
    model = UKBB_All_Substructs_Net2_Binary(
        hidden_channels = 512,
        num_node_features = 3, 
        num_sub_structs = len(substructs),
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    
    # set up callbacks
    early_stop_callback = EarlyStopping(
       monitor='val_loss',
       min_delta=0.00,
       patience=3,
       verbose=False,
       mode='min'
    )

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        save_top_k=1,
        mode='min',
    )
    
    
    #setup trainer
    callbacks_list = [checkpoint_callback]
    
    trainer = pl.Trainer(
        gpus=1, 
        deterministic=True, 
        min_epochs= min_epochs,
        max_epochs = max_epochs,
        callbacks=callbacks_list,
    )
    
    #fit
    model = model.double()
    trainer.fit(model, pl_test)
    model = model.to(device)
    
    # get predictions
    dl = pl_test.test_dataloader()
    true_list = []
    pred_list = []
    model.eval()
    for batch in dl:
        num_graphs = batch["y"].shape[0]
        true = batch["y"].squeeze().detach().cpu().numpy()
        true_list.extend(true)
        x_vals = batch["x"]
        for i, _ in enumerate(x_vals):
            x_vals[i] = x_vals[i].to(model.device)
        preds = model.forward(x_vals, num_graphs).squeeze()
        preds = preds.detach().cpu().numpy()
        pred_list.extend(preds)
        
    my_lists = [true_list, pred_list]
    
    #save predictions
    predictions_name = (predictions_filename_base + str(seed))
    with open(predictions_name, 'wb') as pred_file:
            pickle.dump(my_lists, pred_file)
    
    #save final model
    model_name = (models_filename_base + str(seed))
    with open(model_name, 'wb') as model_file:
            pickle.dump(model, model_file)

Global seed set to 4
  0%|          | 0/1218 [00:00<?, ?it/s]

Initialising underlying dataset... 
...loaded flat_list from ./cached_files/static_oasis_multi_per_sub_cache  with 1740 subjects...
done! 

...rebalancing dataset...
train...


  1%|          | 1/174 [00:00<00:32,  5.31it/s]

done! Now val...


  1%|          | 1/100 [00:00<00:18,  5.27it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Trying larger model (see 256 channels)